<h3>Standard import statements for a MAST Mashup API request</h3>

In [1]:
import sys
import os
import time
import re
import json

try: # Python 3.x
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
except ImportError:  # Python 2.x
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve

try: # Python 3.x
    import http.client as httplib 
except ImportError:  # Python 2.x
    import httplib   

from astropy.table import Table
import numpy as np

import pprint
pp = pprint.PrettyPrinter(indent=4)

<h3>Define the MAST query module to handle appropriate formatting</h3>

In [2]:
def mastQuery(request):
    """Perform a MAST query.
    
        Parameters
        ----------
        request (dictionary): The MAST request json object
        
        Returns head,content where head is the response HTTP headers, and content is the returned data"""
    
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('utf-8')

    # Close the https connection
    conn.close()

    return head,content

<h3>Here is where we begin to customize the code to perform our own queries:</h3>
<p>The first search below sends a filtered query to count all planned observations, designated by a calibration level of -1.  In the 'data' entry, we can see 'Column1' returns with 22133 planned observation entries.</p>
<ul>
    <li>"service":"Mast.Caom.Filtered.TestV230" defines the current test server.</li>
    <li>"paramName":"calib_level" allows us to filter based on calibration level.</li>
    <li>"values":["-1"] defines a planned observation.</li>
    <li>Currently it isn't necessary to filter for JWST since those are the only entries present.</li>
</ul>
<p>Additional parameters to filter on:</p>
<ul>
    <li>target_name</li>
    <li>s_ra</li>
    <li>s_dec</li>
    <li>obs_collection</li>
    <li>instrument</li>
    <li>filters</li>
    <li>proposal_id</li>
</ul>

In [3]:
mashupRequest = {"service":"Mast.Caom.Filtered.TestV230",
                 "format":"json",
                 "params":{
                     "columns":"COUNT_BIG(*)",    # "COUNT_BIG(*)" will only get a count of the results
                     "filters":[
                         {"paramName":"calib_level",
                          "values":["-1"],
                         },
                         {"paramName":"obs_collection",
                          "values":["JWST"]
                         },
                         {"paramName":"proposal_type",
                          "values":["ERS", "GTO"]                             
                         }
                     ]}}
    
headers,outString = mastQuery(mashupRequest)
countData = json.loads(outString)

pp.pprint(countData)

{   'data': [{'Column1': 11918}],
    'fields': [{'name': 'Column1', 'type': 'string'}],
    'msg': '',
    'paging': {   'page': 1,
                  'pageSize': 1,
                  'pagesFiltered': 1,
                  'rows': 1,
                  'rowsFiltered': 1,
                  'rowsTotal': 1},
    'status': 'COMPLETE'}


<h3> Searching by position</h3>
<p>First off, in order to send a filtered position search via the API, we'll need to submit the position in question in degrees.  Converting an ICRS position to degrees is made pretty easy by using the <a href="http://docs.astropy.org/en/stable/coordinates/" target="_blank">astropy.coordinates SkyCoord class</a>.</p>

In [4]:
from astropy.coordinates import SkyCoord

# Use SkyCoord class to convert to degrees
def convert_to_degrees(our_ra, our_dec):
    coords = SkyCoord(our_ra, our_dec)
    ra_deg = coords.ra.deg
    dec_deg = coords.dec.deg
    in_degrees = (ra_deg, dec_deg)
    return in_degrees

# Select our coordinates
RA = '04h16m09.370s'
DEC = '-24d04m20.50s'
SAMPLE_COORDS = convert_to_degrees(RA, DEC)
print(SAMPLE_COORDS)

(64.03904166666666, -24.07236111111111)


<p>Now that we have our RA and Dec available in degrees, we can define a radius (also in degrees) and submit a filtered position query.  In this first case, we keep the "columns":"COUNT_BIG(*)" to simply return a count of the results, in case we hit a large number of entries.</p>

In [5]:
def filtered_position_count(coordinates, radius=0.2):    # radius also in degrees
    ra_deg = coordinates[0]
    dec_deg = coordinates[1]
    mashupRequest = {"service":"Mast.Caom.Filtered.TestV230.Position",
                     "format":"json",
                     "params":{"columns":"COUNT_BIG(*)",    # "COUNT_BIG(*)" will only get a count of the results
                               "filters":[{"paramName":"calib_level",
                                           "values":["-1"]
                                          },
                                          {"paramName":"obs_collection",
                                           "values":["JWST"]
                                          },
                                          {"paramName":"proposal_type",
                                           "values":["ERS", "GTO"]
                                          }],
                               "position":"{0}, {1}, {2}".format(ra_deg, dec_deg, radius)
                              }
                    }

    headers,outString = mastQuery(mashupRequest)
    countData = json.loads(outString)
    data = countData['data']
    count = data[0]['Column1']
    return count

TEST_COUNT = filtered_position_count(SAMPLE_COORDS)
print(TEST_COUNT)

152


<p>In the above result we see we get 164 results, so we can go ahead and submit the full request.</p>

In [6]:
def filtered_position_query(coordinates, radius=0.2):
    ra_deg = coordinates[0]
    dec_deg = coordinates[1]
    mashupRequest = {"service":"Mast.Caom.Filtered.TestV230.Position",
                     "format":"json",
                     "params":{"columns":"*",    # return all fields
                               "filters":[{"paramName":"calib_level",
                                           "values":["-1"]
                                          },
                                          {"paramName":"obs_collection",
                                           "values":["JWST"]
                                          },
                                          {"paramName":"proposal_type",
                                           "values":["ERS", "GTO"]
                                          }],
                               "position":"{0}, {1}, {2}".format(ra_deg, dec_deg, radius)
                              }
                    }

    headers,outString = mastQuery(mashupRequest)
    countData = json.loads(outString)
    return countData

def find_conflicting_targets(our_target, countData):
    data = countData['data']
    ra_target = our_target[0]
    dec_target = our_target[1]
    targets = {}
    
    # Create a dictionary of all unique coordinate pairs along with a count of how many times they
    # are found
    for current in data:
        obsid = current['obsid']
        current_ra = current['s_ra']
        current_dec = current['s_dec']
        current_coords = (current_ra, current_dec)
        if current_coords in targets.keys():
            targets[current_coords] += 1
        else:
            targets[current_coords] = 1

    # For each unique coordinate pair, calculate the distance from the target and display our 
    # results
    for x in sorted(targets.keys()):
        num_obs = targets[x]
        unique_ra = x[0]
        unique_dec = x[1]
        result = "Found {0} planned observations at {1}, {2}".format(num_obs, 
                                                                     unique_ra, 
                                                                     unique_dec)
        distance_ra = abs(unique_ra - ra_target)
        distance_dec = abs(unique_dec - dec_target)
        distance = SkyCoord(distance_ra, distance_dec, frame="icrs", unit='deg')
        if distance_ra < 0.001 and distance_dec < 0.001:    # Account for rounding differences
            result += " (target match)"
        else:
            result += " ({0} away)".format(distance.to_string('hmsdms'))
        print(result)
        
    return targets

TEST_QUERY = filtered_position_query(SAMPLE_COORDS)
TEST_TARGETS = find_conflicting_targets(SAMPLE_COORDS, TEST_QUERY)

Found 13 planned observations at 64.0342, -24.0667138888889 (00h00m01.162s +00d00m20.33s away)
Found 103 planned observations at 64.0390416666667, -24.0723611111111 (target match)
Found 36 planned observations at 64.0416666666667, -24.0661111111111 (00h00m00.63s +00d00m22.5s away)


<h3>Saving results to a file</h3>
<p>This gives us a basic idea of how many observations are currently planned in the vicinity, but we'd now like to examine these observations in more detail.  We can do this by saving the results of our query into a CSV table.</p>

In [7]:
import csv

def write_to_csv_file(countData, filename):
    
    # Column names are stored in the 'fields' dictionary
    fields = countData['fields']
    header = []
    for entry in fields:
        header.append(entry['name'])

    # Use the DictWriter class to write the data dictionary to a .csv file
    directory = os.getcwd()
    filename = directory + "/" + filename
    data = countData['data']
    with open(filename, 'w') as output:
        writer = csv.DictWriter(output, fieldnames=header)
        writer.writeheader()
        for obs in data:
            writer.writerow(obs)
        output.close()
        
    print("Saved {0}".format(filename))
    return filename
    
# Choose a filename for the resulting CSV table
SAVE_AS = 'planned_obs.csv'
SAVED = write_to_csv_file(TEST_QUERY, SAVE_AS)


Saved /Users/pforshay/Documents/1801_plannedobs/planned_obs.csv


<h3>Process multiple targets</h3>
<p>We now have a CSV table with all available parameters of all observations found within a 0.2 degree radius of a set of sample coordinates.  The API allows us to now take this one step further and bring all these modules together and check multiple sets of coordinates back-to-back.</p>

In [ ]:
def check_multiple_targets(coordinates_list, coordinates_format):
    
    # Iterate through our list of coordinate tuples
    for target in coordinates_list:
        print("...checking {0}...".format(target))
        
        # Convert each pair of coordinates to degrees if necessary
        if coordinates_format.lower() == "deg":
            in_degrees = target
        else:
            in_degrees = convert_to_degrees(target[0], target[1])
        
        # Submit an initial count query
        count = filtered_position_count(in_degrees)
        
        # If the count is within a valid range, submit the full query
        if count > 0 and count < 50000:
            query_results = filtered_position_query(in_degrees)
            conflicts = find_conflicting_targets(in_degrees, query_results)
            
            # Generate a filename and write the information to a CSV table
            filename = "results_{0}_{1}.csv".format(target[0], target[1])
            filename = write_to_csv_file(query_results, filename)
            
        # Skip if too many results are found
        elif count > 50000:
            print("More than 50,000 results found!  Please narrow your query.")
            
        # Skip if no results are found
        elif count == 0:
            print("No conflicts found for {0}".format(target))


OUR_COORDINATES_LIST = [('04h16m09.370s', '-24d04m20.50s'),
                        ('05h42m15s', '+48d22m43s'),
                        ('13h36m59.849s', '-29d51m42.97s'),
                        ('20h20m20.20s', '+20d20m20.20s')
                       ]
check_multiple_targets(OUR_COORDINATES_LIST, "icrs")

...checking ('04h16m09.370s', '-24d04m20.50s')...
Found 13 planned observations at 64.0342, -24.0667138888889 (00h00m01.162s +00d00m20.33s away)
Found 103 planned observations at 64.0390416666667, -24.0723611111111 (target match)
Found 36 planned observations at 64.0416666666667, -24.0661111111111 (00h00m00.63s +00d00m22.5s away)
Saved /Users/pforshay/Documents/1801_plannedobs/results_04h16m09.370s_-24d04m20.50s.csv
...checking ('05h42m15s', '+48d22m43s')...
No conflicts found for ('05h42m15s', '+48d22m43s')
...checking ('13h36m59.849s', '-29d51m42.97s')...
No conflicts found for ('13h36m59.849s', '-29d51m42.97s')
...checking ('20h20m20.20s', '+20d20m20.20s')...
No conflicts found for ('20h20m20.20s', '+20d20m20.20s')


<h3>Getting coordinates from target names</h3>
<p>Using the API also gives us access to the name-resolver service, which allows us to input a list of target names instead of coordinates.  The returned coordinates are already in degrees, so we can also skip the convert_to_degrees module.</p>

In [ ]:
def resolve_target_names(target_list):
    
    # Set up an empty list for coordinate results and iterate through the list of target names
    coordinates_list = []
    for target_name in target_list:
        
        # Make a resolver request with the current target name
        resolverRequest = {'service':'Mast.Name.Lookup',
                           'params':{'input':target_name,
                                     'format':'json'}
                          }
        headers, resolvedObjectString = mastQuery(resolverRequest)
        resolvedObject = json.loads(resolvedObjectString)
        
        # If the target name was not found, we will run into IndexErrors when we try to set these 
        # variables
        try:
            target_ra = resolvedObject['resolvedCoordinate'][0]['ra']
            target_dec = resolvedObject['resolvedCoordinate'][0]['decl']
            canonical_name = resolvedObject['resolvedCoordinate'][0]['canonicalName']
        except IndexError:
            print("{0} not found".format(target_name))
            continue
            
        # Add the coordinates as a tuple to the list
        target_coords = (target_ra, target_dec)
        coordinates_list.append(target_coords)
        print("Found {0} at {1}".format(canonical_name, target_coords))
        
    return coordinates_list

OUR_TARGETS_LIST = ['M92',
                    '30 Doradus',
                    'Dumbbell Nebula',
                    'Fake Test Galaxy'
                   ]
NEW_COORDINATES_LIST = resolve_target_names(OUR_TARGETS_LIST)
check_multiple_targets(NEW_COORDINATES_LIST, "deg")

Found MESSIER 092 at (259.28029, 43.13652)
Found RMC 136 at (84.67665, -69.100933)
Found M  27 at (299.901579, 22.721042)
Fake Test Galaxy not found
...checking (259.28029, 43.13652)...
Found 8 planned observations at 259.210094166667, 43.1458638888889 (00h00m16.847s +00d00m33.638s away)
Saved /Users/pforshay/Documents/1801_plannedobs/results_259.28029_43.13652.csv
...checking (84.67665, -69.100933)...


<h3>Searching for moving targets (keyword searches on target_name)</h3>
<p>Moving targets are a bit trickier since we do not have a single set of coordinates to search against.  Instead, we'll look for matching entries in the target_name field and we'll use the freeText paramter with wildcards to find any close matches.  Again, we'll begin with an initial "COUNT_BIG(*)" query to get a results count first.</p>

In [ ]:
def filtered_keyword_count(keyword):
    mashupRequest = {"service":"Mast.Caom.Filtered.TestV230",
                     "format":"json",
                     "params":{"columns":"COUNT_BIG(*)",    # "COUNT_BIG(*)" will only get a count of the results
                               "filters":[{"paramName":"calib_level",
                                           "values":["-1"]
                                          },
                                          {"paramName":"obs_collection",
                                           "values":["JWST"]
                                          },
                                          {"paramName":"proposal_type",
                                           "values":["ERS", "GTO"]
                                          },
                                          {"paramName":"target_name",
                                           "values":[],
                                           "freeText":"%"+keyword+"%"
                                          }]
                              }
                    }

    headers,outString = mastQuery(mashupRequest)
    countData = json.loads(outString)
    data = countData['data']
    count = data[0]['Column1']
    return count

JUPITER_COUNT = filtered_keyword_count("Jupiter")
print(JUPITER_COUNT)

<p>We find 82 target names containing "Jupiter", so we'll go ahead and submit the full query.</p>

In [ ]:
def filtered_keyword_query(keyword):
    mashupRequest = {"service":"Mast.Caom.Filtered.TestV230",
                     "format":"json",
                     "params":{"columns":"*",
                               "filters":[{"paramName":"calib_level",
                                           "values":["-1"]
                                          },
                                          {"paramName":"obs_collection",
                                           "values":["JWST"]
                                          },
                                          {"paramName":"proposal_type",
                                           "values":["ERS", "GTO"]
                                          },
                                          {"paramName":"target_name",
                                           "values":[],
                                           "freeText":"%"+keyword+"%"
                                          }]
                              }
                    }

    headers,outString = mastQuery(mashupRequest)
    countData = json.loads(outString)
    return countData

# Create a list of the unique target names that were returned by the filtered_keyword_query
def keyword_matches(countData):
    data = countData['data']
    targets = []
    for obs in data:
        current_target = obs['target_name']
        targets.append(current_target)
    unique_targets = list(set(targets))
    print("Matched the keyword to: {0}".format(unique_targets))
    return unique_targets
    
JUPITER_QUERY = filtered_keyword_query("Jupiter")
JUPITER_MATCHES = keyword_matches(JUPITER_QUERY)

<p>Copying our multiple targets module from above, with a few tweaks we can now run back-to-back queries on a list of moving target names.  We can also incorporate the write_to_csv_file module to automatically save all our results for further inspection.</p>

In [ ]:
def check_multiple_keywords(keywords_list):
    
    # Iterate through our list of target_name keywords
    for target in keywords_list:
        print("...checking {0}...".format(target))
        
        # Submit an initial count query
        count = filtered_keyword_count(target)
        
        # If the count is within a valid range, submit the full query
        if count > 0 and count < 50000:
            query_results = filtered_keyword_query(target)
            conflicts = keyword_matches(query_results)
            
            # Generate a filename and write the information to a CSV table
            filename = "results_{0}.csv".format(target)
            filename = write_to_csv_file(query_results, filename)
            
        # Skip if too many results are found
        elif count > 50000:
            print("More than 50,000 results found!  Please narrow your query.")
            
        # Skip if no results are found
        elif count == 0:
            print("No conflicts found for {0}".format(target))
            
OUR_MOVING_TARGETS = ["Jupiter",
                      "Ceres",
                      "Titan",
                      "Sun"
                     ]
check_multiple_keywords(OUR_MOVING_TARGETS)